# T5. REFERENCE LEVELS

In this notebook, we extract the reference levels from the t1_semantic segmentation/ t2_instance segmentation

## LIBRARIES

In [1]:
#IMPORT PACKAGES
import os.path
import importlib
from pathlib import Path
import numpy as np
import open3d as o3d
from rdflib import Graph, URIRef, Literal, Namespace

# from tabulate import tabulate
import laspy
import json
from scipy.spatial.transform import Rotation   
import copy

import geomapi
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl
import geomapi.tools.progresstools as pt

#import utils
import context 
import utils as utl
import utils.t5_utils as t5


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

## INPUTS

In [4]:
#paths
path=Path(os.getcwd()).parents[2] # On Onedrive this is 2, on GPU server this is 0

print(path)
input_folder=path/'data'/'t4'/'test' 
# input_folder=path/'data'/'t4'/'train' 

class_file=path/'data'/'_classes.json'
output_folder=path/'data'/'t5'/ 'test'
# output_folder=path/'data'/'t5'/ 'train'

os.makedirs(output_folder, exist_ok=True)

#parameters
threshold_horizontal_clustering=100#m
threshold_vertical_clustering=0.5#m

c:\Users\u0094523\OneDrive - KU Leuven\2024-05 CVPR scan-to-BIM challenge


Import Classes

In [5]:
# Read the JSON file
with open(class_file, 'r') as file:
    json_data = json.load(file)

# Create a dictionary
class_dict = {
    'classes': json_data['classes'],
    'default': json_data['default'],
    'type': json_data['type'],
    'format': json_data['format'],
    'created_with': json_data['created_with']
}
print(class_dict)

{'classes': [{'name': 'unassigned', 'id': 255, 'temp_id': 0, 'color': '#9da2ab'}, {'name': 'floors', 'id': 0, 'temp_id': 1, 'color': '#03c2fc'}, {'name': 'ceilings', 'id': 1, 'temp_id': 2, 'color': '#e81416'}, {'name': 'walls', 'id': 2, 'temp_id': 3, 'color': '#ffa500'}, {'name': 'columns', 'id': 3, 'temp_id': 4, 'color': '#faeb36'}, {'name': 'doors', 'id': 4, 'temp_id': 5, 'color': '#79c314'}, {'name': 'beams', 'id': 5, 'temp_id': 6, 'color': '#79c314'}], 'default': 255, 'type': 'semantic_segmentation', 'format': 'kitti', 'created_with': {'name': 'Saiga', 'version': '1.0.1'}}


## PROCESS LEVELS

In [6]:
files=utl.get_list_of_files(input_folder,'.laz')

for f in files[2:]: 
    
    #import graph
    f_g=Path(f).with_suffix('.ttl')
    print(f_g)
    pcdNodes=tl.graph_path_to_nodes(graphPath=str(f_g))
    
    #import pcd and check if las/pcd variable is already defined    
    print(f'processing {ut.get_filename(f)} ...')      
    las = laspy.read(f) #if 'las' not in globals() else las
    pcd=gmu.las_to_pcd(las) #if 'pcd' not in globals() else pcd # this is the slowest step
        
    #match pcd to nodes
    for c in class_dict['classes']:
        if c['id'] in [0,1]: 
            idx=np.where((las['classes']==c['id']))[0]
            class_pcd=pcd.select_by_index(idx)
            object_labels=las['objects'][idx]
            
            for j in np.unique(object_labels):
                indices=np.where(object_labels==j)[0]
                object_pcd=class_pcd.select_by_index(indices)
                pcdNode=next((x for x in pcdNodes if int(x.object_id) == j), None)
                pcdNode.resource=object_pcd if pcdNode is not None else None
            
    print(f'{len(pcdNodes)} Nodes found')     
    
    #retrieve levelNodes -> CVPR they take highest element in the group instead of the average -> this even should be 20cm higher but this is arbitrary in the CVPR dataset -> 4% error
    floorNodes=[n for n in pcdNodes if n.class_id ==0]
    ceilingNodes=[n for n in pcdNodes if n.class_id ==1]    
    print(f'{len(floorNodes)} floor nodes and {len(ceilingNodes)} ceiling nodes found')
    levelNodes=t5.create_level_nodes((floorNodes+ceilingNodes),threshold_horizontal_clustering=threshold_horizontal_clustering,threshold_vertical_clustering=threshold_vertical_clustering)
    print(f'{len(levelNodes)} levels created at heights {[n.height for n in levelNodes]}')      
    
    #write this information to the 3D detection json
    json_data=t5.levels_to_json(levelNodes,f)
    with open(os.path.join(output_folder,f'{ut.get_filename(f)}_levels.json'), 'w') as file:
        json.dump(json_data, file, indent=4)
    print("JSON data written to file:", os.path.join(output_folder,f'{ut.get_filename(f)}_levels.json') )
    
    #write geometries to file
    joined_references=gmu.join_geometries([n.plane for n in levelNodes])
    success=o3d.io.write_triangle_mesh(filename=os.path.join(output_folder,f'{ut.get_filename(f)}_levels.obj'), mesh=joined_references) 
    print(f' Saving joint references : {success}')
    
    #write graph to file
    graphPath=os.path.join(output_folder,f'{ut.get_filename(f)}_levels.ttl')
    graph=tl.nodes_to_graph(levelNodes,graphPath=graphPath,save=True)


c:\Users\u0094523\OneDrive - KU Leuven\2024-05 CVPR scan-to-BIM challenge\data\t4\test\11_MedOffice_05_F2_small_pred.ttl
processing 11_MedOffice_05_F2_small_pred ...
156 Nodes found
3 floor nodes and 5 ceiling nodes found
2 levels created at heights [0.0161679523, 2.74903721]
JSON data written to file: c:\Users\u0094523\OneDrive - KU Leuven\2024-05 CVPR scan-to-BIM challenge\data\t5\test\11_MedOffice_05_F2_small_pred_levels.json
 Saving joint references : True
c:\Users\u0094523\OneDrive - KU Leuven\2024-05 CVPR scan-to-BIM challenge\data\t4\test\11_MedOffice_05_F4_small_pred.ttl
processing 11_MedOffice_05_F4_small_pred ...
152 Nodes found
8 floor nodes and 9 ceiling nodes found
4 levels created at heights [-4.21099628, 0.0604897509, 2.78621407, 4.15988102]
JSON data written to file: c:\Users\u0094523\OneDrive - KU Leuven\2024-05 CVPR scan-to-BIM challenge\data\t5\test\11_MedOffice_05_F4_small_pred_levels.json
 Saving joint references : True
c:\Users\u0094523\OneDrive - KU Leuven\2024-0

## VISUALIZE

In [7]:
# joined_pcd=gmu.join_geometries([p.resource.paint_uniform_color(ut.random_color()) for p in objectNodes])
# o3d.visualization.draw_geometries([joined_pcd,gmu.sample_geometry(class_pcd)[0]])

In [8]:
# joined_pcd=gmu.join_geometries([n.plane.paint_uniform_color(ut.random_color()) for n in levelNodes if n.resource is not None])
# joined_pcd2=gmu.join_geometries([n.resource.paint_uniform_color(ut.random_color()) for n in (floorNodes+ceilingNodes) if n.resource is not None])
# o3d.visualization.draw_geometries([joined_pcd2]+[n.box for n in levelNodes if n.resource is not None])